In [ ]:
!pip install emoji

In [ ]:
!pip install setfit

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
import emoji
from datasets import Dataset, DatasetDict
import re
import string
import pandas as pd
from sklearn.model_selection import train_test_split
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from sentence_transformers.losses import CosineSimilarityLoss

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Znatno_personal/Отчетность ВКР/Артефакты/Parsing_avito/pure_data.csv')

In [ ]:
df = df.drop('Unnamed: 0', axis=1)

In [ ]:
def convert_emojis_to_words(text):

    # Convert emojis to words
    text = emoji.demojize(text, delimiters=(" ", " "))

    # Remove the : from the words and replace _ with space
    text = text.replace("_", " ")

    return text

In [ ]:
symbols_pattern = re.compile(pattern = "["
    "@_!#$%^&*()<>?/\|}{~√•—"
                       "]+", flags = re.UNICODE) #спецсимволы
# двойные пробелы
space_pattern = re.compile('\s+')

<>:2: DeprecationWarning: invalid escape sequence '\|'
<>:5: DeprecationWarning: invalid escape sequence '\s'
<>:2: DeprecationWarning: invalid escape sequence '\|'
<>:5: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-58-d876ec9db1ec>:2: DeprecationWarning: invalid escape sequence '\|'
  "@_!#$%^&*()<>?/\|}{~√•—"
<ipython-input-58-d876ec9db1ec>:5: DeprecationWarning: invalid escape sequence '\s'
  space_pattern = re.compile('\s+')


In [ ]:
def clear_text(text):
    """ Функция удаления спецсимволов"""
    # удаление спецсимволов и emoji
    pre = symbols_pattern.sub(r'',text)

    return space_pattern.sub(' ', pre)

In [ ]:
def preprocess_text(text):
    """ Финальная функция для обработки """
    # srip + lower + punctuation
    sentence = (
        ''.join([x for x in str(text).strip().lower()])
    )

    return clear_text(sentence)

In [ ]:
df = df.dropna()

In [ ]:
df = df.drop_duplicates()

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df

,message.text,Оценка
0,Опыт работы в косметологии менее года. \nОбяза...,1
1,"Эстетическая, инъекционная, аппаратная космето...",2
2,— Обязанности: не указано.\nКомпания: не указа...,1
3,"Меня зовут Анна, я на данный момент работаю в ...",2
4,"Я открытый и честный человек, который относитс...",2
...,...,...
2522,"Честность, ответственность, прилежность и орга...",10
2523,Твердые профессиональные знания и опыт.\nОтвет...,10
2524,"Эстетическая косметология (массажи, пилинги, ч...",10
2525,"Доброго времени суток, меня зовут Вероника, и ...",10


In [ ]:
df['Оценка'].value_counts()

,count
Оценка,
2,849
1,445
8,253
6,217
10,207
3,203
4,156
7,86
5,79


In [ ]:
df['Оценка'] = df['Оценка'].apply(lambda x: 9 if x > 9 else x)

In [ ]:
df['Оценка'] = df['Оценка'] - 1

In [ ]:
df['message.text'] = df['message.text'].apply(preprocess_text)

In [ ]:
df, df_test = train_test_split(df, test_size=0.1, random_state=42, stratify = df['Оценка'])

In [ ]:
df_train, df_eval = train_test_split(df, test_size=0.1, random_state=42, stratify = df['Оценка'])

In [ ]:
x_test = df_test['message.text']
y_test = df_test['Оценка']

In [ ]:
df_train['Оценка'].value_counts()

,count
Оценка,
1,687
0,361
7,205
8,193
5,176
2,165
3,126
6,69
4,64


In [ ]:
train_dataset = Dataset.from_pandas(df_train)
eval_dataset = Dataset.from_pandas(df_eval)
test_dataset = Dataset.from_pandas(df_test)

In [ ]:
# Simulate the few-shot regime by sampling 64 examples per class
train_dataset = sample_dataset(train_dataset, label_column="Оценка", num_samples=64)

In [ ]:
# Load a SetFit model from Hub
num_classes = 9
model = SetFitModel.from_pretrained(
    "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
     labels = [0,1,2,3,4,5,6,7,8]
)


config.json:   0%|          | 0.00/645 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.89k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


In [ ]:
args = TrainingArguments(
    output_dir="//content/drive/MyDrive/SetFit2/",
    batch_size=64,
    num_epochs=1,
    body_learning_rate = 1e-6,
    use_amp = True,
    eval_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    logging_steps=500,
    evaluation_strategy="steps",
    load_best_model_at_end=True,
    logging_dir = "//content/drive/MyDrive/SetFit/logs",
    report_to = "tensorboard",
)

The `evaluation_strategy` argument is deprecated and will be removed in a future version. Please use `eval_strategy` instead.


In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    metric="accuracy",
    column_mapping={"message.text": "text", "Оценка": "label"}  # Map dataset columns to text/label expected by trainer
)

Applying column mapping to the training dataset
Applying column mapping to the evaluation dataset


Map:   0%|          | 0/2046 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

***** Running training *****
  Num unique pairs = 3421618
  Batch size = 64
  Num epochs = 1


Step,Training Loss,Validation Loss
500,0.254200,0.222230
1000,0.228600,0.193064
1500,0.205900,0.174022
2000,0.189700,0.161747
2500,0.182100,0.152543
3000,0.173100,0.141450
3500,0.165500,0.133351
4000,0.155400,0.127187
4500,0.145400,0.123630
5000,0.136800,0.121195


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
metrics = trainer.evaluate()
metrics

***** Running evaluation *****


{'accuracy': 0.746031746031746}

In [ ]:
pred = model.predict(test_dataset['message.text'])

In [ ]:
print('Accuracy:', accuracy_score(test_dataset['Оценка'], pred))
print('F1 score:', f1_score(test_dataset['Оценка'], pred, average='weighted'))

Accuracy: 0.5816993464052288
F1 score: 0.5935834981169478
